In [1]:
import numpy as np
import pandas as pd
import os
from my_utils.model_selection import BlockingTimeSeriesSplit


metadata = pd.read_csv("../stock_data_cleaned/symbol_sector.csv")
data_path = "../stock_data_model/"
# Define the sectors to keep
sectors_to_keep = ["Finance", "Technology"]

In [16]:
metadata.loc[metadata["Symbol"] == "AAPL", "Sector"].values[0]

'Technology'

In [20]:
for index, row in metadata.iterrows():
    stockID = row["Symbol"]
    sector = row["Sector"]
    if sector in sectors_to_keep and not os.path.exists(data_path + stockID + ".csv"):
        print(stockID)

In [14]:
metadata.iterrows()

<generator object DataFrame.iterrows at 0x000002A3B54E0580>

In [6]:
for index, row in metadata.iterrows():
    stockID = row["Symbol"]
    sector = row["Sector"]
    if os.path.exists(data_path + stockID + ".csv") and sector not in sectors_to_keep:
        os.remove(data_path + stockID + ".csv")

In [ ]:
# 定义时间序列拆分器
tscv = BlockingTimeSeriesSplit(n_splits=5)
# 初始化一个 100x5 的数组，初始值为 0
split_array = np.zeros((100, 5))

# 进行时间序列拆分
for fold, (train_index, test_index) in enumerate(tscv.split(np.arange(100))):
    split_array[train_index, fold] = 1
    split_array[test_index, fold] = 2

In [2]:
original_path = "../stock_data_model/"
split_path = "../stock_data_model_split/"
if not os.path.exists(split_path):
    os.mkdir(split_path)

In [5]:
# 定义时间序列拆分器
tscv = BlockingTimeSeriesSplit(n_splits=5)

for index, row in metadata.iterrows():
    stockID = row["Symbol"]
    sector = row["Sector"]
    # if sector not in sectors_to_keep:
    #     continue
    data = pd.read_csv(original_path + stockID + ".csv")
    split_array = np.zeros((len(data), 5))
    for fold, (train_index, test_index) in enumerate(tscv.split(np.arange(len(data)))):
        split_array[train_index, fold] = 1
        split_array[test_index, fold] = 2
    split_df = pd.DataFrame(
        split_array, columns=["split_0", "split_1", "split_2", "split_3", "split_4"]
    )
    data = pd.concat([data, split_df], axis=1)
    data.to_csv(split_path + stockID + ".csv")